In [2]:
# !pip install spacy
# !pip install gensim
# !python -m spacy download pt_core_news_sm

In [3]:
import numpy as np
from spacy import load
from gensim.models import KeyedVectors
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [4]:
w2v_modelo_cbow = KeyedVectors.load_word2vec_format('/content/drive/MyDrive/curso_word2vec_treinamento/modelo_cbow.txt')
w2v_modelo_sg = KeyedVectors.load_word2vec_format('/content/drive/MyDrive/curso_word2vec_treinamento/modelo_sg.txt')

In [5]:
dados_treino = pd.read_csv('/content/drive/MyDrive/curso_word2vec_treinamento/treino.csv')
dados_teste = pd.read_csv('/content/drive/MyDrive/curso_word2vec_treinamento/teste.csv')

In [6]:
#  disable disabilita os passos da pipeline
nlp = load('pt_core_news_sm', disable=['paser','near','tagger','textcat'])

def tokenizador(texto):
  # Remove não alphanumericos 
  # Remove stop-words
  # Transforma em minuscula

  doc = nlp(texto)
  tokens_validos = []

  for token in doc:
    e_valido = not token.is_stop and token.is_alpha
    if e_valido:
      tokens_validos.append(token.text.lower())

  return tokens_validos

tokens = tokenizador("python / é um 445.255  boa ** linguagem de programacao")
print(tokens)

['python', 'linguagem', 'programacao']


In [7]:
# Faz a soma vetorial de de todas os token da frase
def combinacao_de_vetores_por_soma(palavras, modelo):

  vetor_resultante = np.zeros(300)

  for pn in palavras:
    try:
      vetor_resultante += modelo.get_vector(pn)
    except KeyError:
      pass

  return vetor_resultante

vetor_texto = combinacao_de_vetores_por_soma(tokens, w2v_modelo_cbow)
print(len(vetor_texto))

300


In [8]:
def matriz_vetores(textos, modelo):
  x = len(textos)
  y = 300

  matriz = np.zeros((x,y))

  for i in range(x):
    palavras = tokenizador(textos.iloc[i])
    matriz[i] = combinacao_de_vetores_por_soma(palavras, modelo)

  return matriz

### Utilizando CBOW

In [9]:
matriz_vetores_treino_cbow = matriz_vetores(dados_treino.title, w2v_modelo_cbow)
matriz_vetores_teste_cbow = matriz_vetores(dados_teste.title, w2v_modelo_cbow)

print(matriz_vetores_treino_cbow.shape)
print(matriz_vetores_teste_cbow.shape)

(90000, 300)
(20513, 300)


In [10]:
def classificador(x_treino, y_treino, x_teste, y_teste):
  RL = LogisticRegression(max_iter=800)
  RL.fit(x_treino, y_treino)

  predicoes = RL.predict(x_teste)

  print(classification_report(y_teste, predicoes))

  return RL

RL_cbow = classificador(
    matriz_vetores_treino_cbow, 
    dados_treino.category,
    matriz_vetores_teste_cbow,
    dados_teste.category
)

              precision    recall  f1-score   support

     colunas       0.80      0.71      0.75      6103
   cotidiano       0.64      0.80      0.71      1698
     esporte       0.93      0.87      0.89      4663
   ilustrada       0.13      0.83      0.22       131
     mercado       0.83      0.78      0.81      5867
       mundo       0.75      0.83      0.79      2051

    accuracy                           0.79     20513
   macro avg       0.68      0.80      0.70     20513
weighted avg       0.82      0.79      0.80     20513



### Utilizando SG

In [12]:
matriz_vetores_treino_sg = matriz_vetores(dados_treino.title, w2v_modelo_sg)
matriz_vetores_teste_sg = matriz_vetores(dados_teste.title, w2v_modelo_sg)

print(matriz_vetores_treino_sg.shape)
print(matriz_vetores_teste_sg.shape)

(90000, 300)
(20513, 300)


In [15]:
RL_sg = classificador(
    matriz_vetores_treino_sg, 
    dados_treino.category,
    matriz_vetores_teste_sg,
    dados_teste.category
)

              precision    recall  f1-score   support

     colunas       0.81      0.71      0.76      6103
   cotidiano       0.64      0.80      0.71      1698
     esporte       0.94      0.87      0.90      4663
   ilustrada       0.14      0.89      0.24       131
     mercado       0.84      0.79      0.82      5867
       mundo       0.75      0.84      0.79      2051

    accuracy                           0.79     20513
   macro avg       0.69      0.82      0.70     20513
weighted avg       0.82      0.79      0.80     20513



### Dsiponibilizando o modelo

In [16]:
import pickle

with open('modelo_cbow.pkl', 'wb') as model_file:
  pickle.dump(RL_cbow, model_file)

with open('modelo_sg.pkl', 'wb') as model_file:
  pickle.dump(RL_sg, model_file)

### Utilizando o modelo

In [28]:
frase_1 = "Flamengo é um time de futebol"

tokens_frase_1 = tokenizador(frase_1)
vetor_frase_1 = combinacao_de_vetores_por_soma(tokens_frase_1, w2v_modelo_cbow)

RL_cbow.predict([vetor_frase_1])

array(['esporte'], dtype=object)